In [ ]:
from vistiq.core import StackProcessor, StackProcessorConfig
from vistiq.preprocess import DoGConfig, DoG
from vistiq.seg import OtsuThresholdConfig, OtsuThreshold, LocalThresholdConfig, LocalThreshold
from vistiq.seg import LabellerConfig, Labeller
from vistiq.seg import RegionAnalyzerConfig, RegionAnalyzer, RegionFilterConfig, RegionFilter, RangeFilterConfig, RangeFilter
from vistiq.seg import SegmenterConfig, Segmenter
from vistiq.utils import ArrayIteratorConfig, ArrayIterator, masks_to_labels
from vistiq.analysis import CoincidenceDetectorConfig, CoincidenceDetector

import numpy as np
import stackview

In [ ]:
path = "~/Documents/SDS_/projects/Siegrist/Microsam_Segmentation"

In [ ]:
import ipywidgets as widgets
from pathlib import Path
import os
from bioio import BioImage
from vistiq.utils import load_image

# Expand the path from cell 1
base_path = Path(path).expanduser()

# Common image file extensions
IMAGE_EXTENSIONS = {'.tif', '.tiff', '.png', '.jpg', '.jpeg', '.bmp', '.gif', '.nd2', 
                    '.czi', '.lif', '.ome.tif', '.ome.tiff', '.zarr', '.svs', '.vsi'}

# Recursively get list of all image files in the directory and subdirectories
file_list = []
if base_path.exists() and base_path.is_dir():
    # Use rglob to recursively search for all files
    all_files = sorted(base_path.rglob('*'))
    # Filter to only image files (not directories) and get relative paths
    file_list = [str(f.relative_to(base_path)) for f in all_files 
                 if f.is_file() and f.suffix.lower() in IMAGE_EXTENSIONS]
    if not file_list:
        file_list = ["No image files found"]
else:
    file_list = ["Directory not found"]

# Create file dropdown widget
file_dropdown = widgets.Dropdown(
    options=file_list,
    value=file_list[0] if file_list else None,
    description='File:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create scene dropdown widget (will be updated when file is selected)
scene_dropdown = widgets.Dropdown(
    options=[],
    description='Scene:',
    disabled=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create channel selector widget (will be updated when file or scene is selected)
# Using SelectMultiple to allow selection of multiple channels
channel_dropdown = widgets.SelectMultiple(
    options=[],
    description='Channels:',
    disabled=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='150px')
)

# Initialize img variable in global namespace
img = None
metadata = None



In [ ]:
def update_channels(file_path_str, scene_index=None):
    """Update channel dropdown based on file and scene selection."""
    # Disable channel dropdown while loading
    channel_dropdown.disabled = True
    channel_dropdown.options = ["Loading..."]
    
    try:
        # Use bioio to get available channels
        reader = BioImage(str(file_path_str))
        
        # Set scene if provided
        if scene_index is not None:
            reader.set_scene(scene_index)
        
        # Try to get channel information
        # Check if reader has channel information
        if hasattr(reader, 'channel_names') and reader.channel_names:
            channel_names = reader.channel_names
            if isinstance(channel_names, (list, tuple)):
                channel_options = [f"Channel {i}: {name}" if name else f"Channel {i}" 
                                 for i, name in enumerate(channel_names)]
            else:
                num_channels = len(channel_names) if hasattr(channel_names, '__len__') else 1
                channel_options = [f"Channel {i}" for i in range(num_channels)]
        elif hasattr(reader, 'dims') and 'C' in reader.dims:
            # Try to get channel dimension from xarray
            try:
                xdata = reader.xarray_dask_data
                if hasattr(xdata, 'dims') and 'C' in xdata.dims:
                    num_channels = xdata.dims['C']
                    channel_options = [f"Channel {i}" for i in range(num_channels)]
                else:
                    channel_options = ["Channel 0 (only channel)"]
            except:
                channel_options = ["Channel 0 (only channel)"]
        else:
            # Try to get shape and infer channels
            try:
                test_data = reader.get_image_data()
                if test_data is not None and hasattr(test_data, 'shape'):
                    # Check if last dimension might be channels (typically for RGB)
                    if test_data.ndim >= 3 and test_data.shape[-1] in [1, 3, 4]:
                        num_channels = test_data.shape[-1]
                        channel_options = [f"Channel {i}" for i in range(num_channels)]
                    else:
                        channel_options = ["Channel 0 (only channel)"]
                else:
                    channel_options = ["Channel 0 (only channel)"]
            except:
                channel_options = ["Channel 0 (only channel)"]
        
        channel_dropdown.options = channel_options
        # For SelectMultiple, value should be a tuple/list, default to first channel if available
        channel_dropdown.value = (channel_options[0],) if channel_options else ()
        channel_dropdown.disabled = False
        
    except Exception as e:
        channel_dropdown.options = [f"Error: {str(e)}"]
        channel_dropdown.disabled = True


In [ ]:
def update_scenes(change):
    """Update scene dropdown when file selection changes."""
    selected_relative_path = change['new']
    
    # Check if we have a valid selection
    if not selected_relative_path or selected_relative_path in ["Directory not found", "No image files found"]:
        scene_dropdown.options = []
        scene_dropdown.disabled = True
        channel_dropdown.options = []
        channel_dropdown.disabled = True
        return
    
    if not base_path.exists() or not base_path.is_dir():
        scene_dropdown.options = ["Directory not found"]
        scene_dropdown.disabled = True
        channel_dropdown.options = []
        channel_dropdown.disabled = True
        return
    
    # Construct full path by joining base_path with relative path
    file_path = base_path / selected_relative_path
    
    # Disable scene dropdown while loading
    scene_dropdown.disabled = True
    scene_dropdown.options = ["Loading..."]
    
    try:
        # Use bioio to get available scenes
        reader = BioImage(str(file_path))
        scenes = reader.scenes
        
        # Handle None or empty scenes
        if scenes is None:
            scene_dropdown.options = ["No scenes found (scenes is None)"]
            scene_dropdown.disabled = True
            update_channels(str(file_path), None)
            return
        
        # Check if scenes is iterable and has length
        try:
            num_scenes = len(scenes)
        except (TypeError, AttributeError):
            scene_dropdown.options = ["No scenes found (cannot get length)"]
            scene_dropdown.disabled = True
            update_channels(str(file_path), None)
            return
        
        if num_scenes > 0:
            if num_scenes == 1:
                # If only one scene, show it clearly
                scene_options = ["Scene 0 (only scene)"]
            else:
                # Multiple scenes
                scene_options = [f"Scene {i}" for i in range(num_scenes)]
            scene_dropdown.options = scene_options
            scene_dropdown.value = scene_options[0]
            scene_dropdown.disabled = False
            
            # Update channels for the first scene
            update_channels(str(file_path), 0)
        else:
            scene_dropdown.options = ["No scenes found"]
            scene_dropdown.disabled = True
            update_channels(str(file_path), None)
    except Exception as e:
        scene_dropdown.options = [f"Error: {str(e)}"]
        scene_dropdown.disabled = True
        channel_dropdown.options = []
        channel_dropdown.disabled = True


In [ ]:
def update_channels_on_scene_change(change):
    """Update channels when scene selection changes."""
    selected_relative_path = file_dropdown.value
    selected_scene = change['new']
    
    # Check if we have valid selections
    if not selected_relative_path or selected_relative_path in ["Directory not found", "No image files found"]:
        return
    
    if not selected_scene or selected_scene in ["No scenes found", "Loading..."] or selected_scene.startswith("Error:"):
        return
    
    # Extract scene index
    try:
        scene_parts = selected_scene.split()
        if len(scene_parts) >= 2:
            scene_str = scene_parts[1]
            scene_index = int(scene_str)
            file_path = base_path / selected_relative_path
            update_channels(str(file_path), scene_index)
    except (ValueError, IndexError):
        pass


In [ ]:
# Link file dropdown to scene update - this will trigger update_scenes whenever value changes
file_dropdown.observe(update_scenes, names='value')

# Link scene dropdown to channel update - this will trigger update_channels_on_scene_change when scene changes
scene_dropdown.observe(update_channels_on_scene_change, names='value')

# Initial update to populate scenes and channels for the first file
if file_list and file_list[0] not in ["Directory not found", "No image files found"]:
    update_scenes({'new': file_list[0], 'old': None, 'owner': file_dropdown, 'type': 'change'})

# Create load button
load_button = widgets.Button(
    description='Load Scene',
    disabled=False,
    button_style='info',
    tooltip='Load the selected scene from the selected file',
    icon='check',
    layout=widgets.Layout(width='200px')
)

# Output widget to display status messages
status_output = widgets.Output()

def load_scene(button):
    """Load the selected scene from the selected file into img variable."""
    with status_output:
        status_output.clear_output()
        
        # Get selected file and scene
        selected_relative_path = file_dropdown.value
        selected_scene = scene_dropdown.value
        
        # Get selected channel
        selected_channel = channel_dropdown.value
        
        # Validate selections
        if not selected_relative_path or selected_relative_path in ["Directory not found", "No image files found"]:
            print("Error: Please select a valid file")
            return
        
        # Check if selected_scene is None or invalid
        if selected_scene is None:
            print("Error: No scene selected")
            return
        
        # Check if scene is a string (should always be)
        if not isinstance(selected_scene, str):
            print(f"Error: Scene selection is not a string: {type(selected_scene)}")
            return
        
        # Check if scene is in invalid states
        invalid_scenes = ["No scenes found", "Loading..."]
        try:
            is_invalid = any(invalid in selected_scene for invalid in invalid_scenes) or selected_scene.startswith("Error:")
        except (TypeError, AttributeError):
            print(f"Error: Cannot check scene validity for: {selected_scene}")
            return
        
        if is_invalid:
            print(f"Error: Invalid scene selection: {selected_scene}")
            return
        
        # Construct full path
        file_path = base_path / selected_relative_path
        
        try:
            # Extract scene index from "Scene X" format
            # Handle both "Scene 0" and "Scene 0 (only scene)" formats
            scene_parts = selected_scene.split()
            if len(scene_parts) < 2:
                print(f"Error: Cannot parse scene from '{selected_scene}'")
                return
            
            scene_str = scene_parts[1]  # Get the number part
            scene_index = int(scene_str)
            
            # Extract channel indices if channels are selected
            options = {}
            channel_indices = []
            if selected_channel and len(selected_channel) > 0:
                # selected_channel is now a tuple/list of selected channel strings
                for channel_str in selected_channel:
                    if channel_str.startswith("Error:") or channel_str == "Loading...":
                        continue
                    try:
                        # Extract channel index from "Channel X" or "Channel X: name" format
                        channel_parts = channel_str.split()
                        if len(channel_parts) >= 2:
                            channel_idx_str = channel_parts[1].rstrip(':')
                            channel_idx = int(channel_idx_str)
                            channel_indices.append(channel_idx)
                    except (ValueError, IndexError):
                        pass
            
            # If channels are selected, handle channel selection
            if channel_indices:
                # Check if channels are contiguous
                sorted_indices = sorted(channel_indices)
                is_contiguous = all(sorted_indices[i+1] - sorted_indices[i] == 1 for i in range(len(sorted_indices)-1))
                
                if len(channel_indices) == 1:
                    # Single channel: use slice to keep dimension
                    options['C'] = slice(channel_indices[0], channel_indices[0] + 1)
                    r = load_image(str(file_path), scene_index=scene_index, options=options, squeeze=True)
                    globals()['img'] = r[0]
                    globals()["metadata"] = r[1]
                elif is_contiguous:
                    # Contiguous channels: use slice
                    min_channel = min(channel_indices)
                    max_channel = max(channel_indices)
                    options['C'] = slice(min_channel, max_channel + 1)
                    r = load_image(str(file_path), scene_index=scene_index, options=options, squeeze=True)
                    globals()['img'] = r[0]
                    globals()["metadata"] = r[1]
                else:
                    # Non-contiguous channels: load all channels first, then select specific ones
                    # Load without channel restriction to get all channels
                    r = load_image(str(file_path), scene_index=scene_index, squeeze=False)
                    all_channels = r[0]
                    globals()["metadata"] = r[1]
                    # Find the channel dimension (typically the first dimension after spatial dims)
                    # For bioio, channels are usually in a specific dimension - try to find it
                    # Common patterns: CZYX, TCZYX, etc.
                    # We'll try to find dimension with size >= max(channel_indices) + 1
                    channel_dim = None
                    for dim_idx in range(all_channels.ndim):
                        if all_channels.shape[dim_idx] > max(channel_indices):
                            channel_dim = dim_idx
                            break
                    
                    if channel_dim is not None:
                        # Select the specific channels using advanced indexing
                        # Create index array for the channel dimension
                        indices = [slice(None)] * all_channels.ndim
                        indices[channel_dim] = sorted_indices
                        globals()['img'] = all_channels[tuple(indices)]
                    else:
                        # Fallback: assume channels are in first dimension if shape matches
                        if all_channels.shape[0] > max(channel_indices):
                            globals()['img'] = all_channels[sorted_indices]
                        else:
                            # Couldn't determine channel dimension, load with range
                            min_channel = min(channel_indices)
                            max_channel = max(channel_indices)
                            options['C'] = slice(min_channel, max_channel + 1)
                            r = load_image(str(file_path), scene_index=scene_index, options=options, squeeze=True)
                            globals()['img'] = r[0]
                            globals()["metadata"] = r[1]
            else:
                # No channels selected, load all
                r = load_image(str(file_path), scene_index=scene_index)
                globals()['img'] 
                globals()["metadata"]
            
            # Get img from globals for printing
            img = globals()['img']
            metadata = globals()["metadata"]
            
            if channel_indices:
                if len(channel_indices) == 1:
                    print(f"✓ Loaded Scene {scene_index}, Channel {channel_indices[0]} from {selected_relative_path}")
                else:
                    print(f"✓ Loaded Scene {scene_index}, Channels {channel_indices} from {selected_relative_path}")
            else:
                print(f"✓ Loaded Scene {scene_index} from {selected_relative_path}, metadata={metadata}")
            
            print(f"  Image shape: {img.shape}")
            print(f"  Image dtype: {img.dtype}")
            print(f"  Image metadata: {metadata}")
            
        except ValueError as e:
            print(f"Error: Invalid scene index format: {str(e)}")
            import traceback
            traceback.print_exc()
        except Exception as e:
            print(f"Error loading scene: {str(e)}")
            import traceback
            traceback.print_exc()

# Link button to load function
load_button.on_click(load_scene)


In [ ]:
# Display widgets
display(widgets.VBox([
    file_dropdown, 
    scene_dropdown, 
    channel_dropdown,
    load_button,
    status_output
]))


In [ ]:
dogc = DoGConfig(sigma_low=1.0, sigma_high=12.0)
dogc

In [ ]:
dpn = img[1][40:50]
edu = img[0][40:50]

dog = DoG(dogc)
ddpn = dog.run(dpn)
dedu = dog.run(edu)
print (ddpn.shape)
stackview.slice(np.concatenate([ddpn, dedu], axis=-1), continuous_update=True)

In [ ]:
from vistiq.seg import MicroSAMSegmenterConfig, MicroSAMSegmenter

volume_it_cfg = ArrayIteratorConfig(slice_def=(-3, -2, -1))

filters = [RangeFilter(RangeFilterConfig(attribute="volume", range=(65,5000)))]
ra = RegionAnalyzer(RegionAnalyzerConfig(iterator_config=volume_it_cfg, output_type="dataframe", 
                           properties=["centroid", "volume", "solidity", "aspect_ratio", "sphericity"]))
rf = RegionFilter(RegionFilterConfig(filters=filters))

In [ ]:
dmask, dlabels, dresults = MicroSAMSegmenter(MicroSAMSegmenterConfig(region_filter=rf)).run(ddpn)
dmask = np.array(dmask)
dlabels = np.array(dlabels)
print (f"mask.shape={dmask.shape}, labels.shape={dlabels.shape}")

In [ ]:
rac = RegionAnalyzerConfig(iterator_config=volume_it_cfg, output_type="dataframe", 
                           properties=["centroid", "bbox", "area", "solidity", "aspect_ratio", "circularity"])
ra = RegionAnalyzer(rac)
rn = ra.run(dlabels, metadata={"scale":(1,1,1)})

In [ ]:
rm = ra.run(dlabels, metadata=metadata)

In [ ]:
rn.describe()

In [ ]:
rm.describe()

In [ ]:
emask, elabels, eresults = MicroSAMSegmenter(MicroSAMSegmenterConfig()).run(dedu)
emask = np.array(emask)
elabels = np.array(elabels)
print (f"mask.shape={emask.shape}, labels.shape={elabels.shape}")

In [ ]:
oc = CoincidenceDetectorConfig(method="dice", mode="outline", iterator_config=volume_it_cfg, threshold=0.1)
ovl_results = CoincidenceDetector(oc).run(dlabels, elabels, ("Dpn", "EDU"))

In [ ]:
ovl_results[0]

In [ ]:
print (len(ovl_results[1]["Dpn"]), len(np.unique(dlabels)), np.min(dlabels), np.max(dlabels))
ovl_results[1]["Dpn"]

In [ ]:
import numpy as np
stackview.slice(np.concat([dlabels, elabels], axis=-1))

In [ ]:
import napari
from napari.utils.colormaps.colormap_utils import label_colormap
viewer = napari.Viewer()

In [ ]:
viewer.add_image(dpn, colormap="red", blending="additive", scale=metadata["used_scale"])
viewer.add_image(edu, colormap="green", blending="additive", scale=metadata["used_scale"])
viewer.add_labels(dlabels, features=ovl_results[1]["Dpn"], blending="additive", scale=metadata["used_scale"])
viewer.add_labels(elabels, features=ovl_results[1]["EDU"], blending="additive", scale=metadata["used_scale"])

In [ ]:
dpn_df = ovl_results[1]["Dpn"]
edu_pos = dpn_df[dpn_df["EDU +"]]
edu_neg = dpn_df[~dpn_df["EDU +"]]
plus_mask = np.isin(dlabels, edu_pos.index.values)
minus_mask = np.isin(dlabels, edu_neg.index.values)

labels = masks_to_labels([plus_mask, minus_mask])
viewer.add_labels(labels, name="Dpn Edu+/Edu-", features=ovl_results[1]["Dpn"], scale=metadata["used_scale"])
#viewer.add_image(np.isin(dlabels, edu_pos.index.values), name=f"edu +", blending="additive", colormap="cyan", opacity=0.5, scale=metadata["used_scale"]) 
#viewer.add_image(np.isin(dlabels, edu_neg.index.values), name=f"edu -", blending="additive", colormap="magenta", opacity=0.5, scale=metadata["used_scale"]) 

In [ ]:
#mask, labels, results = MicroSAMSegmenter(MicroSAMSegmenterConfig()).run(img[:,30:35,...])

In [ ]:
bboxes=[((1,2,3),(4,5,6)), ((3,4,5),(6,7,8))] #z1, y1, x1, z2, y2, x2
bboxes=np.array(bboxes)
ndim = bboxes.ndim
bbox_array = bboxes.reshape((len(bboxes),2,-1))
union_mins = np.min(bbox_array[:, 0, :], axis=0)
union_maxs = np.max(bbox_array[:, 1, :], axis=0)

In [ ]:

print (union_mins, union_maxs)

In [ ]:
ndim